Our task is to build a consultant bot that can answer questions of different domains, such as medical with a doctor bot or legal with a lawyer bot.

We will show how flexible ``Component`` and the ``Sequential`` container is to build the same task
in different ways.

1. **Single Task**: We can build a single task where it deals with multiple generators and handles any coding logic.
2. **Multiple Tasks** and combine them using ``Sequential`` which resembles the concept of `Chain` or pipelines in other libraries.


First, lets prepare the imports and prompt templates using `jinjia2` template. We plan to demonstrate how we can use different models too. If this tutorial is the first thing you read, no need to care about more details, but focus on how the `development process` looks like using `LightRAG` library.

In [1]:
import re
from lightrag.core import Component, Generator, Sequential
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [2]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""
template_law = r"""<SYS> You are a lawyer </SYS> User: {{input_str}}"""
template_router = r"""<SYS> You are a router who will route a user question to the right generator.
            Here are your choices in form of key: value pairs:
             {% for key, value in choices.items() %}
                {{ key }}: {{ value }}
             {% endfor %}
            Output the key of your choice.
            </SYS> User question: {{input_str}}
            You:
            """

Let's turn on the library log to help with debugging.

In [3]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [4]:
#Toy example

class DocQA(Component):
    def __init__(self):
        super(DocQA, self).__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        return self.doc(prompt_kwargs={"input_str": query}).data
    

In [5]:
doc = DocQA()
doc

2024-06-09 13:56:51 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
)

In [6]:
# list other subcomponents

for subcomponent in doc.named_components():
    print(subcomponent)

('', DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
))
('doc', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
))
('doc.system_prompt', Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str']))
('doc.model_client', OpenAIClient())


Let's add a parameter

In [20]:
from lightrag.core.parameter import Parameter

doc.register_parameter("demo", param=Parameter(data="demo"))

param Parameter: demo


In [8]:
# list all parameters
for param in doc.named_parameters():
    print(param)

('demo', Parameter: demo)


In [21]:
doc.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'doc': {'type': 'Generator',
    'data': {'_components': {'model_client': {'type': 'OpenAIClient',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        'sync_client': <openai.OpenAI at 0x11b561f50>,
        'async_client': None,
        '_api_key': None}},
      'system_prompt': {'type': 'Prompt',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        '_template_string': '<SYS> You are a doctor </SYS> User: {{input_str}}',
        'prompt_variables': ['input_str'],
        'preset_prompt_kwargs': None}}},
     '_parameters': {},
     'training': False,
     'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
     'preset_prompt_kwargs': None,
     'model_kwargs': {'model': 'gpt-3.5-turbo'},
     'output_processors': None,
     '_trainable_params': []}}},
  '_parameters': {'demo': {'data': 'demo', 'requires_opt': True}},
  'training': False}}

In [24]:
from utils.serialization import save_json

save_json(doc.to_dict(), "doc.json")

In [22]:
doc.state_dict()

OrderedDict([('demo', Parameter: demo)])

In [9]:
doc.call("What is the best treatment for a cold?")

2024-06-09 13:56:51 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for a cold?'}
2024-06-09 13:56:51 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 13:56:51 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for a cold?'}]}
2024-06-09 13:56:53 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 13:56:53 - INFO - [generator.py:205:call] - output: GeneratorOutput(data="The best treatment for a cold typically involves plenty of rest, staying hydrated, and taking over-the-counter medications to help relieve symptoms like aching, sneezing, and coughing. You can also try using steam inhalation, saline nasal spray, and throat lozenges to help alleviate congestion and sore throat. If your symptoms persist or worsen

"The best treatment for a cold typically involves plenty of rest, staying hydrated, and taking over-the-counter medications to help relieve symptoms like aching, sneezing, and coughing. You can also try using steam inhalation, saline nasal spray, and throat lozenges to help alleviate congestion and sore throat. If your symptoms persist or worsen, it's best to consult with a healthcare provider for further evaluation and treatment options."

In [27]:
from lightrag.core.component import FunComponent

def add_one(x):
    return x + 1

fun_component = FunComponent(add_one)
print(fun_component(1))  
print(type(fun_component))  

# output:
# 2
# <class 'core.component.FunComponent'>

2
<class 'core.component.FunComponent'>


In [29]:
from lightrag.core.component import fun_to_component 

fun_component = fun_to_component(add_one)
print(fun_component(1))
print(type(fun_component))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [30]:
# use it as a decorator
@fun_to_component
def add_one(x):
    return x + 1

print(add_one(1))
print(type(add_one))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [33]:
from lightrag.core.component import Sequential

@fun_to_component
def enhance_query(query:str) -> str:
    return query + "Please be concise and only list the top treatments."

seq = Sequential(enhance_query, doc)

query = "What is the best treatment for headache?"
print(seq(query))

2024-06-09 14:42:57 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?Please be concise and only list the top treatments.'}
2024-06-09 14:42:57 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 14:42:57 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?Please be concise and only list the top treatments.'}]}
2024-06-09 14:42:58 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 14:42:58 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='1. Over-the-counter pain relievers like acetaminophen, ibuprofen, or aspirin\n2. Rest and relaxation\n3. Stay hydrated and drink plenty of water', error=None, raw_response='1. Over-the-counter pain relievers like acetaminophen, ibuprofen, o

In [34]:
seq

Sequential(
  (0): EnhanceQueryComponent()
  (1): DocQA(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
      (model_client): OpenAIClient()
    )
  )
)

Here is our first approach to build a single task with multiple generators and call each conditionally.

In [10]:

class ChatBotWithRouter(Component):
    def __init__(self):
        super().__init__()
        model_1_kwargs = {
            "model": "gpt-3.5-turbo",
        }
        model_2_kwargs = {"model": "llama3-8b-8192"}
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs=model_2_kwargs,
        )
        self.router_choices = {
            "doctor": self.create_generator_signature(self.doc),
            "lawyer": self.create_generator_signature(self.lawyer),
            "other": "Choose me the question does not apply to other choices.",
        }
        print(self.router_choices)

        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )

    def call(self, query: str) -> str:
        choice = self.router(
            prompt_kwargs={"input_str": query, "choices": self.router_choices}
        ).data
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Choose me the question does not apply to other choices."

    def create_generator_signature(self, generator: Generator):
        template = generator.template
        pattern = r"<SYS>(.*?)</SYS>"

        matches = re.findall(pattern, template)
        for match in matches:
            print("Content between <SYS> tags:", match)
            return match

Initiate the task component, and print the task details.

In [11]:
task = ChatBotWithRouter()
task

2024-06-09 13:56:53 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


2024-06-09 13:56:53 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
Content between <SYS> tags:  You are a doctor 
Content between <SYS> tags:  You are a lawyer 
{'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}
2024-06-09 13:56:53 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['choices', 'input_str']


ChatBotWithRouter(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
    

Call the task with a query

In [12]:
query = "I have a legal question"
print(task(query))

2024-06-09 13:56:53 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}}
2024-06-09 13:56:53 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 13:56:53 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor:  You are a doctor \n                lawyer:  You are a lawyer \n                other: Choose me the question does not apply to other choices.\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 13:56:53 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https:

Now, let's separate this into multiple subtasks and ``chain`` them using the ``Sequential`` container.

First, the router task which will takes a dictionary of choices and return the selected key. In addition, we use ``_extra_repr`` to improve the default string representation of the task.

As ``Sequential`` will pass the output of one task to the next using positional arguments, we return whatever is needed to the next task in a dictionary.

In [13]:
from typing import Dict
class Router(Component):
    def __init__(self, choices: Dict[str, str] = {}):
        super().__init__()
        self.choices = choices
        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        prompt_kwargs = {"input_str": query, "choices": self.choices}
        choice =  self.router(prompt_kwargs=prompt_kwargs).data
        return {"choice": choice, "query": query}
    
    def _extra_repr(self):
        return f"Choices: {self.choices}, "

See the structure of router task.

In [14]:
r = Router()
r

2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['choices', 'input_str']


Router(
  Choices: {}, 
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
                  </SYS> User question: {{input_str}}
                  You:
                  , prompt_variables: ['choices', 'input_str']
    )
    (model_client): OpenAIClient()
  )
)

Now, lets build another subtask which handles the chat depending on the selected key from the router task.
As the router task returns a dictionary, we will make our input dictionary type that parses the ``choice`` and ``query`` key value pairs.

In [15]:
class Chat(Component):
    def __init__(self):
        super().__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs={"model": "llama3-8b-8192"},
        )
    # to chain together just to make sure the output can be directly passed to the next as input
    def call(self, input: Dict[str, str]) -> Dict[str, str]:
        choice = input.get("choice", None)
        query = input.get("query", None)
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Sorry, I am not able to help you with that."



In [16]:
chat = Chat()
chat

2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


Chat(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
)

Now, lets chain the router and the chat task using the ``Sequential`` container into a runnable pipeline.

In [17]:
class QAWithRouter(Component):
    def __init__(self):
        super().__init__()
        self.router = Router(choices={"doctor": "Doctor", "lawyer": "Lawyer", "other": "Other"})
        self.chat = Chat()
        self.pipeline = Sequential(self.router, self.chat)

    def call(self, query: str) -> str:
        return self.pipeline(query)

In [18]:
qa = QAWithRouter()
qa

2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['choices', 'input_str']


2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 13:56:54 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


QAWithRouter(
  (router): Router(
    Choices: {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}, 
    (router): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (system_prompt): Prompt(
        template: <SYS> You are a router who will route a user question to the right generator.
                    Here are your choices in form of key: value pairs:
                     {% for key, value in choices.items() %}
                        {{ key }}: {{ value }}
                     {% endfor %}
                    Output the key of your choice.
                    </SYS> User question: {{input_str}}
                    You:
                    , prompt_variables: ['choices', 'input_str']
      )
      (model_client): OpenAIClient()
    )
  )
  (chat): Chat(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, pr

In [19]:
qa("I have a legal question")

2024-06-09 13:56:54 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}}
2024-06-09 13:56:54 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 13:56:54 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor: Doctor\n                lawyer: Lawyer\n                other: Other\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 13:56:54 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 13:56:54 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='lawyer',

"Hello! As your lawyer, I'm here to help answer your legal question. Could you please provide a bit more context or information about the issue you're facing? What kind of legal matter are you dealing with?"

# a router and two generators
# every single component might need a signature.
# TODO: LLM for single choices